 Detecton2 kütüphanesi ve gereken kütüphaneler indirmek kullanan bir code parçası

In [1]:
!pip install pyyaml==5.1

import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 4.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Cloning into 'detectron2'...
remote: Enumerating objects: 15303, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 15303 (delta 9), reused 11 (delta 1), pack-reused 15275
Receiving objects: 100% (15303/15303), 6.18 MiB | 10.93 MiB/s, done.
Resolving deltas: 100% (11124/11124), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.8 MB/s eta 0:00:00
  Pr

 gereken kütüphaneler tanımlamak kullanan bir code parçası

In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

#Tanılan görüntü hangi sınıfın olduğunu belirler

In [4]:

from detectron2.structures import BoxMode
from PIL import Image

def get_balloon_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_region_data.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        record = {}

        filename = os.path.join(img_dir, v["filename"])
        if not os.path.exists(filename):
            print(f"File not found: {filename}")
            continue  # or handle the missing file in an appropriate way

        height, width = cv2.imread(filename).shape[:2]
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
        annos = v["regions"]
        objs = []
        for _, anno in annos.items():
            category = anno["region_attributes"]
            shape = anno["shape_attributes"]
            px = shape["x"]
            py = shape["y"]
            pwidth = shape["width"]
            pheight = shape["height"]
            pxn = [px, px+pwidth ]
            pyn = [py, py+pheight]
            if("Happy" == category["Name"]):
              categoryid = 0
            elif("Sad" == category["Name"]):
              categoryid = 1
            elif("Anger" == category["Name"]):
              categoryid = 2
            elif("Fear" == category["Name"]):
              categoryid = 3
            elif("Surprise" == category["Name"] or "surprise" == category["Name"] ):
              categoryid = 4
            obj = {
                "bbox": [np.min(pxn), np.min(pyn), np.max(pxn), np.max(pyn)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": categoryid,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

for d in ["val"]:
  DatasetCatalog.register("Face_" +d, lambda d=d: get_balloon_dicts("/content/drive/MyDrive/data/" + d))
MetadataCatalog.get("Face_val").thing_classes = ["Happy","Sad","Anger","Fear","Surprise"]
MetadataCatalog.get("Face_train").thing_classes = ["Happy","Sad","Anger","Fear","Surprise"]
balloon_metadata = MetadataCatalog.get("Face_val")

 Modul tanımlamak için kullanan bir code parçası

In [5]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
#cfg.MODEL.DEVICE = "cpu"
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")) #faster_rcnn_R_50_FPN_3x
cfg.DATASETS.TRAIN = ("Face_train",)
cfg.DATASETS.TEST = ()
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # faster_rcnn_R_50_FPN_3x
cfg.SOLVER.STEPS = ()  # Adjust these values as needed        # do not decay learning rate
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
cfg.SOLVER.IMS_PER_BATCH = 8  # Number of images per batch
cfg.SOLVER.BASE_LR = 0.0001  # Learning rate
cfg.SOLVER.MAX_ITER = 14000  # Maximum number of training iterations
cfg.SOLVER.STEPS = (6000, 9000)   # Learning rate steps, adjust as needed
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  # RoI heads batch size per image

# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.


cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/drive/MyDrive/Faster RCNN/model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set a custom testing threshold

predictor = DefaultPredictor(cfg)

[01/13 16:02:41 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/Faster RCNN/model_final.pth ...


Çok ifadeli yüz ifade tanıması yada Çok yüzlü ifade tanıması için kullanan code

In [ ]:
import cv2


imo = cv2.imread("65.png")
outputso = predictor(imo)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
print(outputso)
vo = Visualizer(imo[:, :, ::-1],
              metadata=balloon_metadata,
              scale=2.5,
             # instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
)

outo= vo.draw_instance_predictions(outputso["instances"].to("cpu"))
cv2_imshow(outo.get_image()[:, :, ::-1])

Çok ölçekli yüz ifade tanıması için kullanan code

In [ ]:
import cv2
from detectron2.utils.visualizer import Visualizer, ColorMode

# Assuming class_mapping is defined somewhere in your code
class_mapping = {
    0: "Happy",
    1: "Sad",
    2: "Anger",
    3: "Fear",
    4: "Surprise",
}

imo = cv2.imread("re.png")
outputso = predictor(imo)
print(outputso)
# Get the highest predicted class for the entire image
highest_confidence_index = outputso["instances"].get("scores").argmax().item()

highest_confidence_class = class_mapping[outputso["instances"].get("pred_classes")[highest_confidence_index].item()]

print("Highest Confidence Class for the Entire Image:", highest_confidence_class)

# Create a new Instances object for the highest predicted class
highest_class_instances = outputso["instances"][highest_confidence_index : highest_confidence_index + 1].to("cpu")

# Visualize the result with only the highest confidence class
vo = Visualizer(imo[:, :, ::-1], metadata=balloon_metadata, scale=0.5)
outo = vo.draw_instance_predictions(highest_class_instances)
cv2_imshow(outo.get_image()[:, :, ::-1])
